In [2]:
import json
import asyncio

from datetime import datetime
from ..models.models import PriceData
from ..services.database import db
from ..services.trading import strategy
from ..core.config import settings


class BinanceWebsocket:
    def __init__(self):
        self.ws_url = f"wss://stream.binance.com:9443/ws/{settings.TRADING_PAIR.lower()}@bookTicker"
        self.is_connected = False
        self.reconnect_delay = 1
        
    async def handle_message(self, message):
        data = json.loads(message)
        price_data = PriceData(
            timestamp=datetime.utcnow(),
            symbol=settings.TRADING_PAIR,
            price=float(data['b']),  # Best bid price
            quantity=float(data['B'])  # Best bid quantity
        )
        
        # Save price data
        await db.save_price_data(price_data)
        
        # Generate and execute trading signals
        signal = await strategy.generate_signal(settings.TRADING_PAIR, price_data.price)
        if signal:
            await strategy.execute_signal(signal)
            
    async def connect(self):
        while True:
            try:
                async with websockets.connect(self.ws_url) as websocket:
                    self.is_connected = True
                    self.reconnect_delay = 1  # Reset delay on successful connection
                    
                    while True:
                        try:
                            message = await websocket.recv()
                            await self.handle_message(message)
                        except websockets.ConnectionClosed:
                            break
                            
            except Exception as e:
                print(f"WebSocket error: {str(e)}")
                self.is_connected = False
                
                # Exponential backoff for reconnection
                await asyncio.sleep(self.reconnect_delay)
                self.reconnect_delay = min(self.reconnect_delay * 2, 60)  # Max delay of 60 seconds
                
    async def start(self):
        await self.connect()
        
    def is_healthy(self):
        return self.is_connected

ImportError: attempted relative import with no known parent package

In [6]:
import ssl
import websockets
import json

async def connect_to_binance():
    ws_url = "wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1s"

    
    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS)
    async with websockets.connect(ws_url, ssl=ssl_context) as websocket:
        print("Connected to Binance")
        while True:
            try:
                message = await websocket.recv()
                data = json.loads(message)
                print(data)
            except websockets.ConnectionClosed:
                break


async def test():
    ws_url = "wss://stream.binance.com:9443/stream?streams=btcusdt@kline_5s"
    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS)

    async with websockets.connect(ws_url, ssl=ssl_context) as websocket:
        print("Successfully connected to Binance WebSocket")
        
    while True:
        try:
            message = await websocket.recv()
            print(message)
        except websockets.ConnectionClosed:
            print("WebSocket connection closed")
            break

await connect_to_binance()

/tmp/ipykernel_38694/547783249.py:9: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS)


Connected to Binance
{'stream': 'btcusdt@kline_1s', 'data': {'e': 'kline', 'E': 1737383249017, 's': 'BTCUSDT', 'k': {'t': 1737383248000, 'T': 1737383248999, 's': 'BTCUSDT', 'i': '1s', 'f': 4436000247, 'L': 4436000255, 'o': '107593.96000000', 'c': '107593.96000000', 'h': '107593.96000000', 'l': '107593.95000000', 'v': '0.04991000', 'n': 9, 'x': True, 'q': '5370.01450790', 'V': '0.04634000', 'Q': '4985.90410640', 'B': '0'}}}
{'stream': 'btcusdt@kline_1s', 'data': {'e': 'kline', 'E': 1737383250030, 's': 'BTCUSDT', 'k': {'t': 1737383249000, 'T': 1737383249999, 's': 'BTCUSDT', 'i': '1s', 'f': 4436000256, 'L': 4436000264, 'o': '107593.96000000', 'c': '107593.96000000', 'h': '107593.96000000', 'l': '107593.95000000', 'v': '0.03717000', 'n': 9, 'x': True, 'q': '3999.26748350', 'V': '0.03620000', 'Q': '3894.90135200', 'B': '0'}}}
{'stream': 'btcusdt@kline_1s', 'data': {'e': 'kline', 'E': 1737383251014, 's': 'BTCUSDT', 'k': {'t': 1737383250000, 'T': 1737383250999, 's': 'BTCUSDT', 'i': '1s', 'f':

CancelledError: 

In [8]:
test = {"stream":"btcusdt@kline_1s","data":{"e":"kline","E":1737374706015,"s":"BTCUSDT","k":{"t":1737374705000,"T":1737374705999,"s":"BTCUSDT","i":"1s","f":4434768340,"L":4434768559,"o":"108479.49000000","c":"108491.72000000","h":"108491.73000000","l":"108479.49000000","v":"0.27423000","n":220,"x":True,"q":"29749.77109880","V":"0.26946000","Q":"29232.29641450","B":"0"}}}


In [9]:
test["data"]

{'e': 'kline',
 'E': 1737374706015,
 's': 'BTCUSDT',
 'k': {'t': 1737374705000,
  'T': 1737374705999,
  's': 'BTCUSDT',
  'i': '1s',
  'f': 4434768340,
  'L': 4434768559,
  'o': '108479.49000000',
  'c': '108491.72000000',
  'h': '108491.73000000',
  'l': '108479.49000000',
  'v': '0.27423000',
  'n': 220,
  'x': True,
  'q': '29749.77109880',
  'V': '0.26946000',
  'Q': '29232.29641450',
  'B': '0'}}